In [12]:
import numpy as np
import pandas as pd

In [13]:
program_dir = "./"
data_dir = program_dir + "Data/"

combined_tract_file = pd.read_csv(data_dir + "combined_tract.csv")
origin_combined_tract_file = combined_tract_file

# Data Cleaning

In [14]:
combined_tract_file

,Id,Id2_x,Geography_x,Estimate; EMPLOYMENT STATUS - Population 16 years and over,Margin of Error; EMPLOYMENT STATUS - Population 16 years and over,Percent; EMPLOYMENT STATUS - Population 16 years and over,Percent Margin of Error; EMPLOYMENT STATUS - Population 16 years and over,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Margin of Error; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force,...,Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races including Some other race,Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races including Some other race,"Estimate; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Margin of Error; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races",Estimate; HISPANIC OR LATINO AND RACE - Total housing units,Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units,Percent; HISPANIC OR LATINO AND RACE - Total housing units,Percent Margin of Error; HISPANIC OR LATINO AND RACE - Total housing units
0,1400000US17031010100,17031010100,"Census Tract 101, Cook County, Illinois",3845,475,3845,(X),2698,406,70.2,...,0.7,0.8,173,114,3.4,2.3,2664,39,(X),(X)
1,1400000US17031010201,17031010201,"Census Tract 102.01, Cook County, Illinois",5605,564,5605,(X),3822,419,68.2,...,0.0,0.4,124,121,1.7,1.7,2984,84,(X),(X)
2,1400000US17031010202,17031010202,"Census Tract 102.02, Cook County, Illinois",2197,330,2197,(X),1368,225,62.3,...,0.4,0.7,83,68,3.2,2.6,1249,34,(X),(X)
3,1400000US17031010300,17031010300,"Census Tract 103, Cook County, Illinois",5190,546,5190,(X),3292,349,63.4,...,0.0,0.5,76,60,1.3,1.0,3278,90,(X),(X)
4,1400000US17031010400,17031010400,"Census Tract 104, Cook County, Illinois",4273,408,4273,(X),2776,314,65.0,...,0.2,0.4,126,82,2.7,1.7,2239,82,(X),(X)
5,1400000US17031010501,17031010501,"Census Tract 105.01, Cook County, Illinois",3456,383,3456,(X),2585,320,74.8,...,0.0,0.8,194,132,5.1,3.3,2580,64,(X),(X)
6,1400000US17031010502,17031010502,"Census Tract 105.02, Cook County, Illinois",3433,514,3433,(X),2277,487,66.3,...,0.6,0.9,62,60,1.6,1.7,1971,71,(X),(X)
7,1400000US17031010503,17031010503,"Census Tract 105.03, Cook County, Illinois",2235,403,2235,(X),1293,261,57.9,...,1.9,3.1,48,48,2.1,2.0,1334,29,(X),(X)
8,1400000US17031010600,17031010600,"Census Tract 106, Cook County, Illinois",5743,508,5743,(X),4187,374,72.9,...,0.0,0.4,252,152,3.6,2.2,3253,80,(X),(X)
9,1400000US17031010701,17031010701,"Census Tract 107.01, Cook County, Illinois",3066,324,3066,(X),2434,275,79.4,...,0.0,0.8,94,66,2.4,1.7,1642,75,(X),(X)


In [15]:
def data_cleaning(dataframe, verbose):
    # Drop NA
    dataframe.dropna(inplace = True)
    
    for col_name in dataframe.columns:
        col = dataframe[col_name]
        # Drop Margin/Id
        if "Margin" in col_name or col_name == "Id" or col_name == "Geography":
            dataframe.drop(col_name,1,inplace = True)
    
        # Drop uncounted data (X)
        elif (col.all() == "(X)") == True:
            if verbose:
                print("col:", col_name)
            dataframe.drop(col_name, 1, inplace = True)
            continue

In [16]:
data_cleaning(combined_tract_file, False)
print(combined_tract_file.columns.size)
print(combined_tract_file.index.size)

398
1535


In [17]:
def features_isDigit(dataframe, verbose, lines_to_print):
    # verbose = 1 : print specific no digit member for every col
    # verbose = 2 && lines_to_print > 0 : print top lines of sorted no-digit dict
    
    col_num = 0
    dict_no_digit_num = dict()
    list_feature_row_char = list()
    sorted_list_no_digit = list()
    sorted_list_num_no_digit = list()
    
    for col_name in dataframe.columns:
        
        # skip Tract names
        if "Geography" in col_name:
            col_num = col_num + 1
            continue
        
        col = dataframe[col_name]
        
        # skip (X)
        if (col.all() == "(X)") == True:
            continue
        
        col = col.astype(str)
        col = col.str.replace(".","")
        col_isDigit = col.str.isdigit()
        no_digit_col = col_isDigit[col_isDigit == False]
        
        # make no digit dict (not contain all digit features)
        if no_digit_col.size > 0 :
            dict_no_digit_num[col_name] = no_digit_col.size
        
        for i in range(no_digit_col.index.size):
            no_digit_row_index = no_digit_col.index[i]
            # make "specific no digit member for every col" list
            temp_tup = (dataframe.columns[col_num], no_digit_row_index, col[no_digit_row_index])
            list_feature_row_char.append(temp_tup)
            if verbose == 1:
                # print specific no digit member for every col
                print(dataframe.columns[col_num], no_digit_row_index, col[no_digit_row_index])
        col_num = col_num + 1
    
    if dict_no_digit_num == {}:
        return list_feature_row_char, [], []
    
    # sort dict by value (no digit number)
    sorted_list_no_digit = sorted(dict_no_digit_num.items(), key = lambda dict_no_digit_num:dict_no_digit_num[1],
                                  reverse=True)
        
    print("maximum no digit member in one column:", max(dict_no_digit_num.values()))
    if verbose == 2 and lines_to_print > 0:
        #print top lines of sorted no-digit dict
        print("sorted no digit member list:")
        for i in range(lines_to_print):
            print(sorted_list_no_digit[i])
    for row in sorted_list_no_digit:
        sorted_list_num_no_digit.append(row[1])
    return list_feature_row_char, sorted_list_no_digit, sorted_list_num_no_digit

In [18]:
def show_specific_rows(list_feature_row_char, feature_list):
    list_specific_rows = []
    list_each_row = []
    
    for feature in feature_list:
        list_each_row = []
        feature_name = feature[0]
        list_each_row.append(feature_name)
        for tup in list_feature_row_char:
            if tup[0] == feature_name:
                list_each_row.append(tup[1])
        print(list_each_row)
        list_specific_rows.append(list_each_row)
    return list_specific_rows
    

# Feature selection

In [19]:
list_feature_row_char_2013, sorted_list_no_digit_2013, sorted_list_num_no_digit_2013 = features_isDigit(
                                                                         combined_tract_file, 
                                                                         verbose = 2, lines_to_print=10)

maximum no digit member in one column: 579
sorted no digit member list:
('Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Families with female householder, no husband present - With related children under 18 years - With related children under 5 years only', 579)
('Estimate; INCOME AND BENEFITS (IN 2013 INFLATION-ADJUSTED DOLLARS) - With cash public assistance income - Mean cash public assistance income (dollars)', 174)
('Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Married couple families - With related children under 18 years - With related children under 5 years only', 164)
('Estimate; INCOME AND BENEFITS (IN 2013 INFLATION-ADJUSTED DOLLARS) - With Supplemental Security Income - Mean Supplemental Security Income (dollars)', 96)
('Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Families with female househol

In [20]:
print(sorted_list_num_no_digit_2013)

[579, 174, 164, 96, 52, 24, 15, 12, 12, 9, 8, 8, 8, 7, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]


In [23]:
no_digit_features_list_2013 = sorted_list_no_digit_2013[-20:]
list_specific_rows_2013 = show_specific_rows(list_feature_row_char_2013, no_digit_features_list_2013)

['Percent; RACE - Race alone or in combination with one or more other races - Total population - Black or African American', 409, 1316, 1317, 1318]
['Percent; RACE - Race alone or in combination with one or more other races - Total population - American Indian and Alaska Native', 409, 1316, 1317, 1318]
['Percent; RACE - Race alone or in combination with one or more other races - Total population - Asian', 409, 1316, 1317, 1318]
['Percent; RACE - Race alone or in combination with one or more other races - Total population - Native Hawaiian and Other Pacific Islander', 409, 1316, 1317, 1318]
['Percent; RACE - Race alone or in combination with one or more other races - Total population - Some other race', 409, 1316, 1317, 1318]
['Percent; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any race)', 409, 1316, 1317, 1318]
['Percent; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any race) - Mexican', 409, 1316, 1317, 1318]
['Percent; HISPANIC

In [24]:
list_specific_rows_2013

[['Percent; RACE - Race alone or in combination with one or more other races - Total population - Black or African American',
  409,
  1316,
  1317,
  1318],
 ['Percent; RACE - Race alone or in combination with one or more other races - Total population - American Indian and Alaska Native',
  409,
  1316,
  1317,
  1318],
 ['Percent; RACE - Race alone or in combination with one or more other races - Total population - Asian',
  409,
  1316,
  1317,
  1318],
 ['Percent; RACE - Race alone or in combination with one or more other races - Total population - Native Hawaiian and Other Pacific Islander',
  409,
  1316,
  1317,
  1318],
 ['Percent; RACE - Race alone or in combination with one or more other races - Total population - Some other race',
  409,
  1316,
  1317,
  1318],
 ['Percent; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any race)',
  409,
  1316,
  1317,
  1318],
 ['Percent; HISPANIC OR LATINO AND RACE - Total population - Hispanic or Latino (of any

In [25]:
print(len(sorted_list_no_digit_2013))
print(len(combined_tract_file.columns))
print(len(combined_tract_file.index))

192
398
1535


In [26]:
useless_rows_2013 = [409, 1316, 1317, 1318]

In [27]:
useless_rows_2013_2 = [409, 1247, 1316, 1317, 1318]

In [28]:
combined_tract_file.iloc[useless_rows_2013_2]

,Id2_x,Geography_x,Estimate; EMPLOYMENT STATUS - Population 16 years and over,Percent; EMPLOYMENT STATUS - Population 16 years and over,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force,Estimate; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Employed,Percent; EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Employed,...,Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Native Hawaiian and Other Pacific Islander alone,Estimate; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Some other race alone,Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Some other race alone,Estimate; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races,Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races,Estimate; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races including Some other race,Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races including Some other race,"Estimate; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races","Percent; HISPANIC OR LATINO AND RACE - Total population - Not Hispanic or Latino - Two or more races - Two races excluding Some other race, and Three or more races",Estimate; HISPANIC OR LATINO AND RACE - Total housing units
409,17031381700,"Census Tract 3817, Cook County, Illinois",0,0,0,-,0,-,0,-,...,-,0,-,0,-,0,-,0,-,0
1247,17031835700,"Census Tract 8357, Cook County, Illinois",161,161,125,77.6,125,77.6,0,0.0,...,0.0,0,0.0,20,8.7,0,0.0,20,8.7,71
1316,17031980000,"Census Tract 9800, Cook County, Illinois",0,0,0,-,0,-,0,-,...,-,0,-,0,-,0,-,0,-,0
1317,17031980100,"Census Tract 9801, Cook County, Illinois",0,0,0,-,0,-,0,-,...,-,0,-,0,-,0,-,0,-,0
1318,17031990000,"Census Tract 9900, Cook County, Illinois",0,0,0,-,0,-,0,-,...,-,0,-,0,-,0,-,0,-,0


In [29]:
def data_selection(dataframe, feature_list = [], row_list = []):
    # remove coloums
    if len(feature_list) > 0 and len(row_list) < 1:
        for feature in feature_list:
            dataframe.drop(feature[0], 1, inplace = True)
    else:
    # remobe rows
        dataframe.drop(dataframe.index[row_list], inplace = True)
        

In [30]:
data_selection(combined_tract_file, row_list=useless_rows_2013)

In [31]:
list_feature_row_char_2013, sorted_list_no_digit_2013, sorted_list_num_no_digit_2013 = features_isDigit(
                                                                         combined_tract_file, 
                                                                         verbose = 0, lines_to_print=10)
len(sorted_list_no_digit_2013)

maximum no digit member in one column: 575


61

In [32]:
print(len(sorted_list_no_digit_2013))
print(len(combined_tract_file.columns))
print(len(combined_tract_file.index))

61
398
1531


In [33]:
data_selection(combined_tract_file, feature_list=sorted_list_no_digit_2013)

In [34]:
print(len(sorted_list_no_digit_2013))
print(len(combined_tract_file.columns))
print(len(combined_tract_file.index))

61
337
1531


In [35]:
list_feature_row_char_2013, sorted_list_no_digit_2013, sorted_list_num_no_digit_2013 = features_isDigit(
                                                                         combined_tract_file, 
                                                                         verbose = 0, lines_to_print=10)
len(sorted_list_no_digit_2013)

0

In [36]:
combined_tract_file.to_csv("./Features/features_2013.csv", sep = "\t")

# Run Data Cleaning for all the data (from 2010 to 2017)

In [37]:
program_dir = "./"
data_dir = program_dir + "Data/"
raw_acs_data_dir = data_dir + "combined_raw/"

combined_tract_file_2010 = pd.read_csv(raw_acs_data_dir + "2010 combined.csv", index_col = 0)
combined_tract_file_2011 = pd.read_csv(raw_acs_data_dir + "2011 combined.csv", index_col = 0)
combined_tract_file_2012 = pd.read_csv(raw_acs_data_dir + "2012 combined.csv", index_col = 0)
combined_tract_file_2013 = pd.read_csv(raw_acs_data_dir + "2013 combined.csv", index_col = 0)
combined_tract_file_2014 = pd.read_csv(raw_acs_data_dir + "2014 combined.csv", index_col = 0)
combined_tract_file_2015 = pd.read_csv(raw_acs_data_dir + "2015 combined.csv", index_col = 0)
combined_tract_file_2016 = pd.read_csv(raw_acs_data_dir + "2016 combined.csv", index_col = 0)
combined_tract_file_2017 = pd.read_csv(raw_acs_data_dir + "2017 combined.csv", index_col = 0)

/Users/mingyu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10,11,14,15,18,19,22,23,26,27,30,31,34,35,38,39,42,43,46,47,50,51,54,55,58,59,62,63,66,67,68,69,74,75,78,79,82,83,86,87,94,95,98,99,106,107,110,111,118,119,122,123,126,127,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,186,187,190,191,194,195,198,199,202,203,206,207,210,211,214,215,218,219,222,223,226,227,230,231,238,239,242,243,246,247,250,251,254,255,258,259,266,267,270,271,274,275,278,279,282,283,286,287,290,291,294,295,298,299,302,303,306,307,310,311,314,315,318,319,322,323,336,337,340,341,344,345,348,349,352,353,356,357,364,365,372,373,376,377,380,381,388,389,396,397,404,405,408,409,412,413,416,417,420,421,424,425,436,437,440,441,444,445,448,449,452,453,460,461,464,465,468,469,472,473,476,477,480,481,484,485,488,489,492,493,496,497,500,501,504,505,508,509,516,517,520,521,524,525,528,529,536,537,540,541,54

/Users/mingyu/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (10,11,14,15,18,19,22,23,26,27,30,31,34,35,38,39,42,43,46,47,50,51,54,55,58,59,62,63,66,67,68,69,74,75,78,79,82,83,86,87,94,95,98,99,106,107,110,111,118,119,122,123,126,127,130,131,134,135,138,139,142,143,146,147,150,151,154,155,158,159,162,163,166,167,170,171,174,175,178,179,182,183,186,187,190,191,194,195,198,199,202,203,206,207,210,211,214,215,218,219,222,223,226,227,230,231,238,239,242,243,246,247,250,251,254,255,258,259,266,267,270,271,274,275,278,279,282,283,286,287,290,291,294,295,298,299,302,303,306,307,310,311,314,315,318,319,322,323,336,337,340,341,344,345,348,349,352,353,356,357,364,365,372,373,376,377,380,381,388,389,396,397,404,405,408,409,412,413,416,417,420,421,424,425,426,427,436,437,440,441,444,445,448,449,452,453,460,461,464,465,468,469,472,473,476,477,480,481,484,485,488,489,492,493,496,497,500,501,504,505,508,509,516,517,520,521,524,525,528,529,536,537,54

In [38]:
list_combined_tract_file = []
list_combined_tract_file.append(combined_tract_file_2010)
list_combined_tract_file.append(combined_tract_file_2011)
list_combined_tract_file.append(combined_tract_file_2012)
list_combined_tract_file.append(combined_tract_file_2013)
list_combined_tract_file.append(combined_tract_file_2014)
list_combined_tract_file.append(combined_tract_file_2015)
list_combined_tract_file.append(combined_tract_file_2016)
list_combined_tract_file.append(combined_tract_file_2017)

In [39]:
# Some stats
for combined_tract_files in list_combined_tract_file:
    print("rows number:",len(combined_tract_files), "columns number:", len(combined_tract_files.columns))

rows number: 1535 columns number: 1475
rows number: 1535 columns number: 1475
rows number: 1535 columns number: 1475
rows number: 1535 columns number: 1487
rows number: 1535 columns number: 1487
rows number: 1535 columns number: 1499
rows number: 1535 columns number: 1499
rows number: 1535 columns number: 1519


In [40]:
# Data Cleaning
for combined_tract_files in list_combined_tract_file:
    data_cleaning(combined_tract_files, False)

In [41]:
# Some stats
for combined_tract_files in list_combined_tract_file:
    print("rows number:",len(combined_tract_files), "columns number:", len(combined_tract_files.columns))

rows number: 1535 columns number: 628
rows number: 1535 columns number: 628
rows number: 1535 columns number: 692
rows number: 1535 columns number: 692
rows number: 1535 columns number: 692
rows number: 1535 columns number: 699
rows number: 1535 columns number: 699
rows number: 1535 columns number: 712


In [42]:
# Data slection
list_feature_row_char_2010, sorted_list_no_digit_2010, sorted_list_num_no_digit_2010 = features_isDigit(
                                                                         combined_tract_file_2010, 
                                                                         verbose = 2, lines_to_print=0)
list_feature_row_char_2011, sorted_list_no_digit_2011, sorted_list_num_no_digit_2011 = features_isDigit(
                                                                         combined_tract_file_2011, 
                                                                         verbose = 2, lines_to_print=0)
list_feature_row_char_2012, sorted_list_no_digit_2012, sorted_list_num_no_digit_2012 = features_isDigit(
                                                                         combined_tract_file_2012, 
                                                                         verbose = 2, lines_to_print=0)
list_feature_row_char_2013, sorted_list_no_digit_2013, sorted_list_num_no_digit_2013 = features_isDigit(
                                                                         combined_tract_file_2013, 
                                                                         verbose = 2, lines_to_print=0)
list_feature_row_char_2014, sorted_list_no_digit_2014, sorted_list_num_no_digit_2014 = features_isDigit(
                                                                         combined_tract_file_2014, 
                                                                         verbose = 2, lines_to_print=0)
list_feature_row_char_2015, sorted_list_no_digit_2015, sorted_list_num_no_digit_2015 = features_isDigit(
                                                                         combined_tract_file_2013, 
                                                                         verbose = 2, lines_to_print=0)
list_feature_row_char_2016, sorted_list_no_digit_2016, sorted_list_num_no_digit_2016 = features_isDigit(
                                                                         combined_tract_file_2016, 
                                                                         verbose = 2, lines_to_print=0)
list_feature_row_char_2017, sorted_list_no_digit_2017, sorted_list_num_no_digit_2017 = features_isDigit(
                                                                         combined_tract_file_2013, 
                                                                         verbose = 2, lines_to_print=0)

maximum no digit member in one column: 660
maximum no digit member in one column: 640
maximum no digit member in one column: 595
maximum no digit member in one column: 579
maximum no digit member in one column: 561
maximum no digit member in one column: 579
maximum no digit member in one column: 1535
maximum no digit member in one column: 579


In [43]:
print("Cols number (containing '-')")
print(len(sorted_list_num_no_digit_2010))
print(len(sorted_list_num_no_digit_2011))
print(len(sorted_list_num_no_digit_2012))
print(len(sorted_list_num_no_digit_2013))
print(len(sorted_list_num_no_digit_2014))
print(len(sorted_list_num_no_digit_2015))
print(len(sorted_list_num_no_digit_2016))
print(len(sorted_list_num_no_digit_2017))

Cols number (containing '-')
297
297
318
318
318
318
321
318


In [44]:
sorted_list_no_digit_2016[-10:]

[('Percent; ANCESTRY - Total population - Russian', 4),
 ('Percent; ANCESTRY - Total population - Scotch-Irish', 4),
 ('Percent; ANCESTRY - Total population - Scottish', 4),
 ('Percent; ANCESTRY - Total population - Slovak', 4),
 ('Percent; ANCESTRY - Total population - Subsaharan African', 4),
 ('Percent; ANCESTRY - Total population - Swedish', 4),
 ('Percent; ANCESTRY - Total population - Swiss', 4),
 ('Percent; ANCESTRY - Total population - Ukrainian', 4),
 ('Percent; ANCESTRY - Total population - Welsh', 4),
 ('Percent; ANCESTRY - Total population - West Indian (excluding Hispanic origin groups)',
  4)]

In [45]:
print("2010:", sorted_list_num_no_digit_2010)
print("2011:", sorted_list_num_no_digit_2011)
print("2012:", sorted_list_num_no_digit_2012)
print("2013:", sorted_list_num_no_digit_2013)
print("2014:", sorted_list_num_no_digit_2014)
print("2015:", sorted_list_num_no_digit_2015)
print("2016:", sorted_list_num_no_digit_2016)
print("2017:", sorted_list_num_no_digit_2017)

2010: [660, 595, 595, 379, 379, 299, 247, 247, 247, 247, 247, 195, 193, 129, 68, 68, 68, 68, 68, 68, 68, 68, 68, 68, 57, 48, 38, 25, 23, 15, 14, 13, 13, 12, 12, 12, 11, 11, 10, 10, 10, 9, 9, 9, 9, 8, 8, 8, 7, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]
2011: [640, 594, 594, 322, 322, 254, 225, 225, 

In [46]:
no_digit_features_list_2010 = sorted_list_no_digit_2010[40:52]
no_digit_features_list_2010_2 = [features for features in sorted_list_no_digit_2010 if features[1]==6]
list_specific_rows_2010 = show_specific_rows(list_feature_row_char_2010, no_digit_features_list_2010_2)

['Estimate; INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS) - Median household income (dollars)', 409, 698, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Unrelated individuals 15 years and over', 400, 409, 1247, 1316, 1317, 1318]
['Estimate; FERTILITY - Per 1,000 women 15 to 50 years old - Per 1,000 women 35 to 50 years old', 404, 409, 1247, 1316, 1317, 1318]
['Percent; SCHOOL ENROLLMENT - Nursery school, preschool', 105, 409, 1247, 1316, 1317, 1318]
['Percent; SCHOOL ENROLLMENT - Kindergarten', 105, 409, 1247, 1316, 1317, 1318]
['Percent; SCHOOL ENROLLMENT - Elementary school (grades 1-8)', 105, 409, 1247, 1316, 1317, 1318]
['Percent; SCHOOL ENROLLMENT - High school (grades 9-12)', 105, 409, 1247, 1316, 1317, 1318]
['Percent; SCHOOL ENROLLMENT - College or graduate school', 105, 409, 1247, 1316, 1317, 1318]


In [47]:
drop_tracts_2010 = [105, 400, 409, 698, 1247, 1316, 1317, 1318]
print(len(drop_tracts_2010))

8


In [48]:
no_digit_features_list_2011 = sorted_list_no_digit_2011[40:52]
no_digit_features_list_2011_2 = [features for features in sorted_list_no_digit_2011 if features[1]==6]
list_specific_rows_2011 = show_specific_rows(list_feature_row_char_2011, no_digit_features_list_2011_2)

['Estimate; INCOME AND BENEFITS (IN 2011 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars)', 396, 409, 1247, 1316, 1317, 1318]
['Estimate; INCOME AND BENEFITS (IN 2011 INFLATION-ADJUSTED DOLLARS) - Median earnings for female full-time, year-round workers (dollars)', 352, 409, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All families - With related children under 18 years', 409, 1194, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Under 18 years', 409, 1194, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Under 18 years - Related children under 18 years', 409, 1194, 1247, 1316, 1317, 1318]
['Percent; SCHOOL ENROLLMENT - Nursery school, preschool', 105, 409, 1247, 1316, 1317, 1318]
['Percent; SCHOOL ENROLLMENT - Kin

In [49]:
drop_tracts_2011 = [105, 352, 396, 409, 1194, 1247, 1316, 1317, 1318]

In [50]:
no_digit_features_list_2012 = sorted_list_no_digit_2012[40:52]
no_digit_features_list_2012_2 = [features for features in sorted_list_no_digit_2012 if features[1]==6]
list_specific_rows_2012 = show_specific_rows(list_feature_row_char_2012, no_digit_features_list_2012_2)

['Percent; EMPLOYMENT STATUS - All parents in family in labor force', 409, 1194, 1247, 1316, 1317, 1318]
['Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)', 408, 409, 1247, 1316, 1317, 1318]
['Estimate; INCOME AND BENEFITS (IN 2012 INFLATION-ADJUSTED DOLLARS) - Median nonfamily income (dollars)', 409, 653, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - All families - With related children under 18 years', 409, 1194, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Under 18 years', 409, 1194, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - Under 18 years - Related children under 18 years', 409, 1194, 1247, 1316, 1317, 1318]


In [51]:
drop_tracts_2012 = [1194, 409, 1247, 1316, 1317, 1318]

In [52]:
no_digit_features_list_2013 = sorted_list_no_digit_2013[40:52]
no_digit_features_list_2013_2 = [features for features in sorted_list_no_digit_2013 if features[1]==6]
list_specific_rows_2013 = show_specific_rows(list_feature_row_char_2013, no_digit_features_list_2013_2)

['Percent; SEX AND AGE - 65 years and over - Male', 400, 409, 1247, 1316, 1317, 1318]
['Percent; SEX AND AGE - 65 years and over - Female', 400, 409, 1247, 1316, 1317, 1318]
['Percent; EMPLOYMENT STATUS - Own children under 6 years - All parents in family in labor force', 409, 478, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - 65 years and over', 400, 409, 1247, 1316, 1317, 1318]
['Percent; DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION - 65 years and over - With a disability', 400, 409, 1247, 1316, 1317, 1318]


In [53]:
drop_tracts_2013 = [400, 478, 409, 1247, 1316, 1317, 1318]

In [54]:
no_digit_features_list_2014 = sorted_list_no_digit_2014[40:52]
no_digit_features_list_2014_2 = [features for features in sorted_list_no_digit_2014 if features[1]==5]
list_specific_rows_2014 = show_specific_rows(list_feature_row_char_2014, no_digit_features_list_2014_2)

['Percent; SEX AND AGE - 65 years and over - Male', 409, 1247, 1316, 1317, 1318]
['Percent; SEX AND AGE - 65 years and over - Female', 409, 1247, 1316, 1317, 1318]
['Percent; EMPLOYMENT STATUS - Own children under 6 years - All parents in family in labor force', 409, 1247, 1316, 1317, 1318]
['Percent; COMMUTING TO WORK - Workers 16 years and over - Car, truck, or van -- drove alone', 409, 1247, 1316, 1317, 1318]
['Percent; COMMUTING TO WORK - Workers 16 years and over - Car, truck, or van -- carpooled', 409, 1247, 1316, 1317, 1318]
['Percent; COMMUTING TO WORK - Workers 16 years and over - Public transportation (excluding taxicab)', 409, 1247, 1316, 1317, 1318]
['Percent; COMMUTING TO WORK - Workers 16 years and over - Walked', 409, 1247, 1316, 1317, 1318]
['Percent; COMMUTING TO WORK - Workers 16 years and over - Other means', 409, 1247, 1316, 1317, 1318]
['Percent; COMMUTING TO WORK - Workers 16 years and over - Worked at home', 409, 1247, 1316, 1317, 1318]
['Estimate; COMMUTING TO W

In [55]:
drop_tracts_2014 = [409, 1247, 1316, 1317, 1318]

In [56]:
no_digit_features_list_2015 = sorted_list_no_digit_2015[40:52]
no_digit_features_list_2015_2 = [features for features in sorted_list_no_digit_2015 if features[1]==6]
list_specific_rows_2015 = show_specific_rows(list_feature_row_char_2015, no_digit_features_list_2015_2)

['Percent; SEX AND AGE - 65 years and over - Male', 400, 409, 1247, 1316, 1317, 1318]
['Percent; SEX AND AGE - 65 years and over - Female', 400, 409, 1247, 1316, 1317, 1318]
['Percent; EMPLOYMENT STATUS - Own children under 6 years - All parents in family in labor force', 409, 478, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - 65 years and over', 400, 409, 1247, 1316, 1317, 1318]
['Percent; DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION - 65 years and over - With a disability', 400, 409, 1247, 1316, 1317, 1318]


In [57]:
drop_tracts_2015 = [62, 400, 409, 1247, 1316, 1317, 1318]

In [58]:
no_digit_features_list_2016 = sorted_list_no_digit_2016[40:52]
no_digit_features_list_2016_2 = [features for features in sorted_list_no_digit_2016 if features[1]==5]
list_specific_rows_2016 = show_specific_rows(list_feature_row_char_2016, no_digit_features_list_2016_2)

['Percent; SEX AND AGE - 65 years and over - Male', 409, 1247, 1316, 1317, 1318]
['Percent; SEX AND AGE - 65 years and over - Female', 409, 1247, 1316, 1317, 1318]
['Percent; EMPLOYMENT STATUS - Own children of the householder under 6 years - All parents in family in labor force', 126, 409, 1316, 1317, 1318]
['Estimate; COMMUTING TO WORK - Mean travel time to work (minutes)', 409, 1247, 1316, 1317, 1318]
['Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Nonfamily households - Mean nonfamily income (dollars)', 409, 1247, 1316, 1317, 1318]
['Estimate; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Median earnings for workers (dollars)', 409, 1247, 1316, 1317, 1318]
['Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population 18 to 64 years - In labor force: - Unemployed: - With health insurance coverage', 112, 409, 1316, 1317, 1318]
['Percent; HEALTH INSURANCE COVERAGE - Civilian noninstitutionalized population 18 to 64 years - In labor

In [59]:
drop_tracts_2016 = [409, 1247, 1316, 1317, 1318]

In [60]:
no_digit_features_list_2017 = sorted_list_no_digit_2017[40:52]
no_digit_features_list_2017_2 = [features for features in sorted_list_no_digit_2017 if features[1]==6]
list_specific_rows_2017 = show_specific_rows(list_feature_row_char_2017, no_digit_features_list_2017_2)

['Percent; SEX AND AGE - 65 years and over - Male', 400, 409, 1247, 1316, 1317, 1318]
['Percent; SEX AND AGE - 65 years and over - Female', 400, 409, 1247, 1316, 1317, 1318]
['Percent; EMPLOYMENT STATUS - Own children under 6 years - All parents in family in labor force', 409, 478, 1247, 1316, 1317, 1318]
['Percent; PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL - 65 years and over', 400, 409, 1247, 1316, 1317, 1318]
['Percent; DISABILITY STATUS OF THE CIVILIAN NONINSTITUTIONALIZED POPULATION - 65 years and over - With a disability', 400, 409, 1247, 1316, 1317, 1318]


In [61]:
drop_tracts_2017 = [400, 409, 1247, 1316, 1317, 1318]

In [62]:
data_selection(combined_tract_file_2010, row_list=drop_tracts_2010)
list_feature_row_char_2010, sorted_list_no_digit_2010, sorted_list_num_no_digit_2010 = features_isDigit(
                                                                         combined_tract_file_2010, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2010, feature_list=sorted_list_no_digit_2010)
red = features_isDigit(combined_tract_file_2010, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 652


In [63]:
data_selection(combined_tract_file_2011, row_list=drop_tracts_2011)
list_feature_row_char_2011, sorted_list_no_digit_2011, sorted_list_num_no_digit_2011 = features_isDigit(
                                                                         combined_tract_file_2011, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2011, feature_list=sorted_list_no_digit_2011)
red = features_isDigit(combined_tract_file_2011, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 633


In [64]:
data_selection(combined_tract_file_2012, row_list=drop_tracts_2012)
list_feature_row_char_2012, sorted_list_no_digit_2012, sorted_list_num_no_digit_2012 = features_isDigit(
                                                                         combined_tract_file_2012, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2012, feature_list=sorted_list_no_digit_2012)
red = features_isDigit(combined_tract_file_2012, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 589


In [65]:
data_selection(combined_tract_file_2013, row_list=drop_tracts_2013)
list_feature_row_char_2013, sorted_list_no_digit_2013, sorted_list_num_no_digit_2013 = features_isDigit(
                                                                         combined_tract_file_2013, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2013, feature_list=sorted_list_no_digit_2013)
red = features_isDigit(combined_tract_file_2013, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 572


In [66]:
data_selection(combined_tract_file_2014, row_list=drop_tracts_2014)
list_feature_row_char_2014, sorted_list_no_digit_2014, sorted_list_num_no_digit_2014 = features_isDigit(
                                                                         combined_tract_file_2014, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2014, feature_list=sorted_list_no_digit_2014)
red = features_isDigit(combined_tract_file_2014, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 556


In [67]:
data_selection(combined_tract_file_2015, row_list=drop_tracts_2015)
list_feature_row_char_2015, sorted_list_no_digit_2015, sorted_list_num_no_digit_2015 = features_isDigit(
                                                                         combined_tract_file_2015, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2015, feature_list=sorted_list_no_digit_2015)
red = features_isDigit(combined_tract_file_2015, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 1528


In [68]:
data_selection(combined_tract_file_2016, row_list=drop_tracts_2016)
list_feature_row_char_2016, sorted_list_no_digit_2016, sorted_list_num_no_digit_2016 = features_isDigit(
                                                                         combined_tract_file_2016, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2016, feature_list=sorted_list_no_digit_2016)
red = features_isDigit(combined_tract_file_2016, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 1530


In [69]:
data_selection(combined_tract_file_2017, row_list=drop_tracts_2017)
list_feature_row_char_2017, sorted_list_no_digit_2017, sorted_list_num_no_digit_2017 = features_isDigit(
                                                                         combined_tract_file_2017, 
                                                                         verbose = 2, lines_to_print=0)
data_selection(combined_tract_file_2017, feature_list=sorted_list_no_digit_2017)
red = features_isDigit(combined_tract_file_2017, verbose = 2, lines_to_print=10)

maximum no digit member in one column: 1529


In [70]:
# Some stats
for combined_tract_files in list_combined_tract_file:
    print("rows number:",len(combined_tract_files), "columns number:", len(combined_tract_files.columns))

rows number: 1527 columns number: 578
rows number: 1526 columns number: 584
rows number: 1529 columns number: 650
rows number: 1528 columns number: 652
rows number: 1530 columns number: 657
rows number: 1528 columns number: 653
rows number: 1530 columns number: 654
rows number: 1529 columns number: 665


In [80]:
combined_tract_file_2010.to_csv("./Features/features_2010.csv")
combined_tract_file_2011.to_csv("./Features/features_2011.csv")
combined_tract_file_2012.to_csv("./Features/features_2012.csv")
combined_tract_file_2013.to_csv("./Features/features_2013.csv")
combined_tract_file_2014.to_csv("./Features/features_2014.csv")
combined_tract_file_2015.to_csv("./Features/features_2015.csv")
combined_tract_file_2016.to_csv("./Features/features_2016.csv")
combined_tract_file_2017.to_csv("./Features/features_2017.csv")

In [72]:
useless_tracts = [409, 1316, 1317, 1318, 1247, 126, 112, 140, 1194, 400, 62, 92, 105]

# Feature intersection

In [73]:
for i in range(len(list_combined_tract_file) - 1):
    print(len(np.intersect1d(list_combined_tract_file[i].columns, list_combined_tract_file[i+1].columns)))

515
515
192
585
566
583
556


In [75]:
# Select shared features
shared_features = list_combined_tract_file[0].columns
for i in range(len(list_combined_tract_file) - 1):
    shared_features = np.intersect1d(shared_features, list_combined_tract_file[i+1].columns)

In [78]:
# Generate shared features tables
list_shared_combined_tract_file = []
for i in range(len(list_combined_tract_file)):
    shared_combined_file = list_combined_tract_file[i][shared_features]
    list_shared_combined_tract_file.append(shared_combined_file)

In [79]:
# Write to csv
for i in range(len(list_shared_combined_tract_file)):
    year = 2010 + i
    list_shared_combined_tract_file[i].to_csv("./Features/features_"+ str(year) + "_shared.csv")